## Consensus Trees from Maximum Likehood Method K2P
Here we will visualize the consensus trees file generated by IQ-TREE, UPGMA and NJ using DendroPy.


In [3]:
%pip install dendropy
%pip install --upgrade pip

  Obtaining dependency information for dendropy from https://files.pythonhosted.org/packages/7c/3b/f880fd07cb79a46ca838e46ba021fcdaa464bb21e2a9264a7afe77046a88/DendroPy-4.6.1-py3-none-any.whl.metadata
  Using cached DendroPy-4.6.1-py3-none-any.whl.metadata (3.8 kB)
Using cached DendroPy-4.6.1-py3-none-any.whl (458 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os

path = os.path.expanduser("~/dataml/iqtree_1.contree")


In [5]:
#importing the libraries 
import dendropy
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [14]:
from ete3 import Tree, TreeStyle

In [15]:
# Load the tree with ETE
t = Tree("/home/beatriz/Leu_Neufische/2_MIC_Prediction/dataml/iqtree_1.contree")

# Define a tree style
ts = TreeStyle()
ts.show_leaf_name = True
ts.show_branch_length = True
ts.show_branch_support = True

# Show the tree
t.show(tree_style=ts)


: 

In [3]:
from ete3 import Tree, TreeStyle, NodeStyle

# Load the tree
tree_1 = Tree("/home/beatriz/Leu_Neufische/2_MIC_Prediction/dataml/filtered_influencer_sequences_1.fasta.treefile")

# Define the tree style
ts = TreeStyle()
ts.show_leaf_name = True
ts.show_branch_length = True
ts.show_branch_support = True

# Save the tree as an image
output_image_path_1 = "/home/beatriz/Leu_Neufische/2_MIC_Prediction/dataml/tree_image_1.png"
tree_1.render(output_image_path_1, tree_style=ts, w=800)

# Display the image in Jupyter
from IPython.display import Image
Image(filename=output_image_path_1)



: 